In [7]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
from scipy.stats import hmean
import matplotlib.pyplot as plt
import os
from collections import Counter

In [8]:
def csv_parser(log_file, results_file):
    results_list = []
    with open(log_file,"r") as fp:
        for line in fp:
            split = line.split()
            split2 = line.split(",")
            if(len(split)>0):
                if("pmpakos@" in split[0]):
                    matrix_name = ''
                    # print("---------")
            if(len(split)>2):
                if((split[0]=="INFO:") & (split[1]=="loading") & (split[2]=="Mtx")):
                    print(split[-1])
                    matrix_name = split[-1].split("/")[-1]
                    matrix_name = matrix_name.replace(".mtx","")
            if(len(split2)>2):
                if(split2[1]!=" matrix"):
                    line=line.replace("app",matrix_name).strip("DATA_CSV:,")
                    # line=line.replace(",","\t")
                    results_list.append(line)
                    print(line)

    file = open(results_file,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [595]:
%%time
def clean_matrix_generation_log2(start_gen):
    fw_list = []
    cnt = 0
    with open(start_gen) as fp:
        prev_line = "NULL"
        param_df = pd.DataFrame()
        param_df["param"] = []
        param_df["name"] = []
        
        for line in fp:
            if(">>>> " in line):
                line = line.strip(">>>> ").replace(" ",",")#.replace(".mtx","")
                line_spl = line.split(",")
                line_spl[0] = line_spl[0].replace(".0_","_")
                line_new = ",".join(line_spl)
                param_df = param_df.append({"param": prev_line, "name": line_spl[0]}, ignore_index = True)
                if(line not in fw_list):
                    fw_list.append(line)
                else:
                    cnt+=1
            prev_line = line.strip("\n")
    print("duplicates found :", cnt)
    return param_df



mem_range = "4-32"
# mem_range = "32-512"
# mem_range = "512-2048"

start_gen = "./generation_stats/normal_"+mem_range+"_log.txt"
param_df = clean_matrix_generation_log2(start_gen)

counter = Counter(param_df["name"])
filtered_counter = dict()
for (key, value) in counter.items():
    if(value > 1):
        filtered_counter[key] = value

filtered_counter = dict(sorted(filtered_counter.items(), key=lambda item: item[1]))
duplicate_matrices = filtered_counter.keys()
print(len(duplicate_matrices))

fw = open("duplicate_matrices_neighbours_generator_"+mem_range+".txt","w")
for matrix in duplicate_matrices:
    fw.write(matrix+"\n")
    lista = list(param_df[param_df["name"] == matrix]["param"])
    for a in lista:
        fw.write('\t'+a+"\n")

duplicates found : 621
478
CPU times: user 5.27 s, sys: 0 ns, total: 5.27 s
Wall time: 5.29 s


In [13]:
def clean_matrix_generation_log(start_gen, results_gen):
    fw = open(results_gen,"w")
    fw_list = []
    cnt = 0
    with open(start_gen) as fp:
        prev_line = "NULL"
        param_df = pd.DataFrame()
        
        for line in fp:
            if(">>>> " in line):
                param_df.append({"param": prev_line, "name": line}, ignore_index = True)
                if(line not in fw_list):
                    fw_list.append(line)
                    line = line.strip(">>>> ").replace(" ",",")#.replace(".mtx","")
                    line_spl = line.split(",")
                    line_spl[0] = line_spl[0].replace(".0_","_")
                    # line_spl[0] = line_spl[0].split("_")
                    # del line_spl[0][3]
                    # line_spl[0] = "_".join(line_spl[0])
                    line_new = ",".join(line_spl)
                    fw.write(line_new)
                else:
                    cnt+=1
                    
            prev_line = line
    print("duplicates found :", cnt)
    fw.close()

def clean_results(start_list, results_list):
    for t1,t2 in zip(start_list,results_list):
        fw = open(t2,"w")
        with open(t1) as fp:
            for line in fp:
                if("DATA_CSV:," in line):
                    line_new = line.strip("DATA_CSV:,").replace(".0_","_")
                    fw.write(line_new)
        fw.close()
    
def read_results_single(result_file):
    header = ["matrix_name","original rows","original cols","original NNZs",
              "padded rows","padded cols","padded NNZs","padding overhead[%]",
              "num of runs","total run time[sec]","time[ms]/run","performance[GFLOPs]","performance (padded)[GFLOPs]","mem_footprint[MB]"]
    df = pd.read_table(result_file, delimiter =",", names=header)
    return df

def read_results(results_list, results_gen, results_csv):
    # this header was used for old result reporting
    # header = ["matrix_name","mem_range","avg_nnz_per_row","std_nnz_per_row","avg_bw","std_bw","avg_sc","std_sc"]
    # v2 dgal generator
    header = ["matrix_name", "distribution", "placement", "seed", 
              "nr_rows", "nr_cols", "nr_nnz", "density", "mem_footprint", "mem_range", 
              "avg_nnz_per_row", "std_nnz_per_row", 
              "avg_bw", "std_bw", "avg_bw_scaled", "std_bw_scaled", 
              "avg_sc", "std_sc", "avg_sc_scaled", "std_sc_scaled", 
              "skew_coeff", "avg_num_neighbours", "cross_row_similarity"]
    df_gen = pd.read_table(results_gen, delimiter =",", names=header) 
    
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    matrices = dataframes[0]["matrix_name"].unique()

    n_runs = len(results_list)
    results_final = []
    cntt=0
    for matrix in matrices:
        # print(matrix)
        selected_gen = df_gen[df_gen["matrix_name"]==matrix]
        if(len(selected_gen)==0):
            print('\t', matrix)
            cntt+=1
        else:
            matrix_name = np.asarray(selected_gen["matrix_name"])[0]
            distribution = np.asarray(selected_gen["distribution"])[0]
            placement = np.asarray(selected_gen["placement"])[0]
            seed = np.asarray(selected_gen["seed"])[0]
            nr_rows = np.asarray(selected_gen["nr_rows"])[0]
            nr_cols = np.asarray(selected_gen["nr_cols"])[0]
            nr_nnz = np.asarray(selected_gen["nr_nnz"])[0]
            density = np.asarray(selected_gen["density"])[0]
            # if I want to show mem_footprint of CSR double representation
            mem_footprint = np.asarray(selected_gen["mem_footprint"])[0]
            
            mem_range = np.asarray(selected_gen["mem_range"])[0]
            avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            std_bw = np.asarray(selected_gen["std_bw"])[0]
            avg_bw_scaled = np.asarray(selected_gen["avg_bw_scaled"])[0]
            std_bw_scaled = np.asarray(selected_gen["std_bw_scaled"])[0]
            avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            std_sc = np.asarray(selected_gen["std_sc"])[0]
            avg_sc_scaled = np.asarray(selected_gen["avg_sc_scaled"])[0]
            std_sc_scaled = np.asarray(selected_gen["std_sc_scaled"])[0]
            skew_coeff = np.asarray(selected_gen["skew_coeff"])[0]
            avg_num_neighbours = np.asarray(selected_gen["avg_num_neighbours"])[0]
            cross_row_similarity = np.asarray(selected_gen["cross_row_similarity"])[0]

            matrix_list = matrix.split("_")
            mtx_name = matrix_list[0]

            # nr_rows = int(matrix_list[1])
            # nr_cols = int(matrix_list[2])

            # if(len(matrix_list)==7):#random
            #     placement = "random"
            #     diagonal_factor = 1
            # else: #diagonal
            #     placement = "diagonal"
            #     diagonal_factor = matrix_list[6]

            # seed = matrix_list[-1].strip("n").strip("g")

            # distr = list(filter(lambda x: x.isalpha(), matrix_list[-1]))[0]
            # if(distr=="g"):
            #     distribution="gamma"
            # else:
            #     distribution="normal"

            # nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
            # density = nr_nnz/(nr_rows*nr_cols)*100

            runtime_iter = np.zeros(shape=(n_runs,1))
            perf_padded = np.zeros(shape=(n_runs,1))
            # mem_footprint = np.zeros(shape=(n_runs,1))

            for run in range(0,n_runs):
                df = dataframes[run]
                runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
                perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
                # mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
            if(perf_padded[0][0]<0): # why? nobody knows why
                continue

            runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
            perf_padded_hm = hmean(perf_padded,axis=0)[0]
            # if I want to show mem_footprint of Xilinx sparse double representation
            # mem_footprint = hmean(mem_footprint,axis=0)[0]

            # selected_gen = df_gen[df_gen["matrix_name"]==matrix]
            # mem_range = np.asarray(selected_gen["mem_range"])[0]
            # avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            # std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            # avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            # std_bw = np.asarray(selected_gen["std_bw"])[0]
            # avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            # std_sc = np.asarray(selected_gen["std_sc"])[0]/nr_cols

            W_avg = 33
            J_estimated = W_avg*runtime_iter_hm

            line_list = [matrix_name, distribution, placement, seed,
                         nr_rows, nr_cols, nr_nnz, density, mem_footprint, mem_range,
                         avg_nnz_per_row, std_nnz_per_row, 
                         avg_bw, std_bw, avg_bw_scaled, std_bw_scaled,
                         avg_sc, std_sc, avg_sc_scaled, std_sc_scaled,
                         skew_coeff, avg_num_neighbours, cross_row_similarity, 
                         "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                        ]
            line = ",".join(str(x) for x in line_list) + "\n"
            results_final.append(line)
        
    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()
    print("Results for",len(results_final),"matrices for",results_csv.split("/")[-1])
    if(cntt>0):
        print(cntt, "matrices not found")

def extract_results_of_distr_memrange(distr_memrange):
    start_gen = "./generation_stats/"+distr_memrange+"_log.txt"
    results_gen = "./generation_stats/"+distr_memrange+"_log_CLEAN.txt"
    clean_matrix_generation_log(start_gen, results_gen)

    start_list = [
        "./dirty/"+distr_memrange+"_run1.txt",
    ]

    results_list = [
        "./clean/"+distr_memrange+"_run_CLEAN1.txt",
    ]
    clean_results(start_list, results_list)

    results_csv = "./results_"+distr_memrange+".csv"
    read_results(results_list, results_gen, results_csv)


In [536]:
%%time

distr_base = "normal"
# mem_ranges = ["4-8","8-16","16-32","32-64","64-128","128-256","256-512","512-1024","1024-2048"]
mem_ranges = ["4-32", "32-512", "512-2048"]

for mem_range in mem_ranges:
    distr_memrange = distr_base + "_" + mem_range
    extract_results_of_distr_memrange(distr_memrange)

filenames = ["results_"+distr_base+"_"+mr+".csv" for mr in mem_ranges]

with open('../Benchmarks/xilinx_spmv_4-2048_dtype-D.csv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

duplicates found : 465
Results for 1690 matrices for results_normal_4-32.csv
duplicates found : 100
Results for 2330 matrices for results_normal_32-512.csv
duplicates found : 0
Results for 1425 matrices for results_normal_512-2048.csv
CPU times: user 37.2 s, sys: 132 ms, total: 37.3 s
Wall time: 44.7 s


In [ ]:
# %%time
# distr_memrange = "validation_matrices_synthetic_small"
# extract_results_of_distr_memrange(distr_memrange)
# filename = "results_"+distr_memrange+".csv"

# with open('../Benchmarks/xilinx_spmv_'+distr_memrange+'_dtype-D.csv', 'w') as outfile:
#     with open(filename) as infile:
#             outfile.write(infile.read())

---

---

In [15]:
def read_results2(results_list, results_csv):
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    # get matrices sorted by mem_footprint and then begin processing them (so that I do not have to sort them later...)
    matrices = dataframes[0]["matrix_name"].unique()
    # matrices = dataframes[0].groupby(["matrix name","mem_footprint[MB]"]).size().reset_index().sort_values("mem_footprint[MB]").reset_index(drop=True)["matrix name"]
    
    n_runs = len(results_list)
    results_final = []
    for matrix in matrices:
        nr_rows = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original rows"])[0]
        nr_cols = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original cols"])[0]        
        nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
        density = nr_nnz/(nr_rows*nr_cols)*100
        runtime_iter = np.zeros(shape=(n_runs,1))
        perf_padded = np.zeros(shape=(n_runs,1))
        mem_footprint = np.zeros(shape=(n_runs,1))

        for run in range(0,n_runs):
            df = dataframes[run]
            runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
            perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
            mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
        if(perf_padded[0][0]<0): # why? nobody knows why
            continue

        runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
        perf_padded_hm = hmean(perf_padded,axis=0)[0]
        mem_footprint = hmean(mem_footprint,axis=0)[0]

        W_avg = 33
        J_estimated = W_avg*runtime_iter_hm
        
        line_list = [matrix, nr_rows, nr_cols, nr_nnz, density, mem_footprint, 
                     "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                    ]
        line = ",".join(str(x) for x in line_list) + "\n"
        results_final.append(line)

    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()

In [ ]:
# start_list = [
#     "./dirty/validation_matrices_run1.txt",
# ]

# results_list = [
#     "./clean/validation_matrices_run1.txt",
# ]
# clean_results(start_list, results_list)

# results_csv = "./results_validation_matrices.csv"
# read_results2(results_list, results_csv)

---

In [602]:
def split_equally(starter_file, nummer, extra):
    cnt_m = sum(1 for line in open(starter_file,"r"))
    print(cnt_m, nummer)
    if(cnt_m%nummer != 0):
        step = cnt_m//(nummer-10-1)
    else:
        step = cnt_m//(nummer-10)
    fp = open(starter_file,"r")
    lines = fp.readlines()
    cnt_file = 0
    for num in range(10, nummer):
        partial_file = starter_file.split(".txt")[0].replace(extra,"") + "_part" + str(num+1) + ".txt"
        fw = open(partial_file,"w")
        cnt_m = 0
        while(cnt_m < step and cnt_file < len(lines)):
            curr_line = lines[cnt_file]
            fw.write(curr_line)
            cnt_file += 1
            cnt_m += 1  # int(curr_line.split(" ")[0])
        fw.close()
        print(partial_file.split("/")[-1], "\t", cnt_file, "\t", cnt_m)
    fp.close()
    
distr_memrange = "4-32"
extra = "extra_1.4_"
starter_file = "../matrix_generation_parameters/synthetic_matrices_small_dataset_"+extra+distr_memrange+".txt"
nummer = 33

split_equally(starter_file, nummer, extra)

720 33
synthetic_matrices_small_dataset_4-32_part11.txt 	 32 	 32
synthetic_matrices_small_dataset_4-32_part12.txt 	 64 	 32
synthetic_matrices_small_dataset_4-32_part13.txt 	 96 	 32
synthetic_matrices_small_dataset_4-32_part14.txt 	 128 	 32
synthetic_matrices_small_dataset_4-32_part15.txt 	 160 	 32
synthetic_matrices_small_dataset_4-32_part16.txt 	 192 	 32
synthetic_matrices_small_dataset_4-32_part17.txt 	 224 	 32
synthetic_matrices_small_dataset_4-32_part18.txt 	 256 	 32
synthetic_matrices_small_dataset_4-32_part19.txt 	 288 	 32
synthetic_matrices_small_dataset_4-32_part20.txt 	 320 	 32
synthetic_matrices_small_dataset_4-32_part21.txt 	 352 	 32
synthetic_matrices_small_dataset_4-32_part22.txt 	 384 	 32
synthetic_matrices_small_dataset_4-32_part23.txt 	 416 	 32
synthetic_matrices_small_dataset_4-32_part24.txt 	 448 	 32
synthetic_matrices_small_dataset_4-32_part25.txt 	 480 	 32
synthetic_matrices_small_dataset_4-32_part26.txt 	 512 	 32
synthetic_matrices_small_dataset_4-3

---

In [615]:
def clean_progress_log(start_gen, clean_gen, flag_write = True):
    if(flag_write):
        fw = open(clean_gen,"w")
    cnt, cnt2 = 0, 0
    with open(start_gen) as fp:
        for line in fp:
            # print(line)
            if( flag_write and ("%|" not in line) and not(line=="\n") and ("MEMORY USED :" not in line) and ("->" not in line)):
                fw.write(line)
            if("MATRIX FINISHED." in line):
                cnt+=1
            if("partition done." in line):
                cnt2+=1
    if(flag_write):
        fw.close()
    return cnt, cnt2
    
def clean_progress_log_wrapper(distr_memrange):
    prefix = "../matrix_generation_parameters/"
    max_ID = 35
    cnt_total = 0
    cnt2_total = 0
    cnt3_total = 0
    for ID in range(1,max_ID,1):
        param_set = prefix+"synthetic_matrices_small_dataset_"+distr_memrange+"_part"+str(ID)+".txt"
        if(os.path.exists(param_set)):
            cnt3 = sum(1 for line in open(param_set,"r"))
            cnt3_total += cnt3
            cnt, cnt2 = 0, 0
            start_gen = prefix+"progress_"+distr_memrange+"_"+str(ID)
            if(os.path.exists(start_gen)):
                clean_gen = prefix+"progress_"+distr_memrange+"_"+str(ID)+"_clean"
                cnt, cnt2 = clean_progress_log(start_gen, clean_gen)
            cnt_total += cnt
            cnt2_total += cnt2
            print(ID,"\t",cnt,'/',cnt3, "ready.\t(", cnt2, "successful )")

    if(cnt_total>0):
        print(cnt_total,"/",cnt3_total,"ready\t\t(",cnt2_total,"successful - success rate", np.round(cnt2_total/cnt_total*100,2),"% )")
    print(cnt3_total-cnt_total, "remain")
    final_log = "../FPGA_stuff/generation_stats/normal_" + distr_memrange + "_log.txt"
    cnt, cnt2 = clean_progress_log(final_log, "", False)
    print("\nFINAL_LOG\n\t",cnt, "ready\t(", cnt2, "successful - success rate", np.round(cnt2/cnt*100,2),"% )")

distr_memrange = "4-32"
clean_progress_log_wrapper(distr_memrange)


0 remain

FINAL_LOG
	 3600 ready	( 3130 successful - success rate 86.94 % )
